<a href="https://colab.research.google.com/github/alelasantillan/gpt2-example/blob/master/First_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Analyticsvidhya - GPT-2 implementation
https://www.analyticsvidhya.com/blog/2019/07/openai-gpt2-text-generator-python/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29

In [9]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 188 (delta 0), reused 0 (delta 0), pack-reused 184
Receiving objects: 100% (188/188), 4.36 MiB | 5.90 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [0]:
import os
os.chdir("gpt-2")


In [0]:
os.getcwd()

'/content/gpt-2'

In [0]:
os.listdir()

['requirements.txt',
 'download_model.py',
 'src',
 'LICENSE',
 'Dockerfile.gpu',
 '.gitattributes',
 '.git',
 'CONTRIBUTORS.md',
 '.gitignore',
 'DEVELOPERS.md',
 'README.md',
 'Dockerfile.cpu']

In [0]:
!pip3 install tensorflow-gpu==1.12.0

In [11]:
#vi que en el ejemplo de https://github.com/avivajpeyi/investigating-gpt2/blob/master/GPT2_Investigation.ipynb
#primero bbajaba el modelo y despues instalaba los requerimientos
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 604kB 21.1MB/s 
     |████████████████████████████████| 51kB 21.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/d4/7d/8e/95d6c85f2080a8be09a12a669474122b2e3921fccd3922baf1
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [1]:
!python3 download_model.py 345M

python3: can't open file 'download_model.py': [Errno 2] No such file or directory


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
#os.chdir("gpt-2")
os.chdir('src')

In [0]:
os.listdir()

['encoder.py',
 'sample.py',
 'interactive_conditional_samples.py',
 '__pycache__',
 'model.py',
 'generate_unconditional_samples.py']

In [0]:
os.getcwd()

'/content/gpt-2/src'

In [0]:
!pip3 install regex

     |████████████████████████████████| 655kB 40.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import sys

file_dir = os.path.dirname("/content/gpt-2/src")
sys.path.append(file_dir)
import model, sample, encoder


In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [0]:
#os.chdir("../")
#os.chdir('src')
#os.getcwd()
#os.listdir()
#os.mkdir('models')
#os.chdir('models')
#os.mkdir('345M')
os.chdir('345M')

In [0]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

FileNotFoundError: ignored

# "2nd try"

In [2]:
import os
#os.chdir("../")
#os.chdir('src')
#os.chdir('content')
#os.getcwd()
os.listdir()
#os.mkdir('models')
#os.chdir('models')
#os.mkdir('345M')
#os.chdir('345M')

['.config', 'gpt-2', 'sample_data']

In [3]:
# testing GPU connection
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found; Go to Runtime > Change Runtime Type > Harware Accelerator > GPU')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
# Clone repo 
! git clone https://github.com/openai/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:

# Move to gpt-2 repo
import os
os.chdir('gpt-2')

In [6]:

# download model and install dependencies
!python download_model.py 345M
!pip3 install -r requirements.txt

Fetching checkpoint: 1.00kit [00:00, 594kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 33.2Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 786kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:28, 50.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 4.34Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 32.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 34.0Mit/s]                                                       


In [7]:
#en https://github.com/avivajpeyi/investigating-gpt2/blob/master/GPT2_Investigation.ipynb
#bajaba esto para unconditional generation
!python3 src/generate_unconditional_samples.py --model_name='345M' --nsamples=3 --top_k=10


W0731 14:01:59.724915 139774885799808 deprecation_wrapper.py:119] From src/generate_unconditional_samples.py:53: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-07-31 14:01:59.726980: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-31 14:01:59.729845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-31 14:01:59.730277: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-07-31 14:01:59.730568: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-31 14:01:59.731951: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Succes

# For conditional generation
Generates text sampels conditional on some user input

Notes on flags:
The code comes with a few flags available, with a default value:

* seed = None || a random value is generated unless specified. give a specific integer value if you want to reproduce same results in the future.
* nsamples = 1 || specify the number of samples you want to print
* length = None || number of tokens (words) to print on each sample.
* batch_size = 1 || how many inputs you want to process simultaneously. doesn't seem to affect the results.
* temperature = 1 || scales logits before sampling prior to softmax.
* top_k = 0 || truncates the set of logits considered to those with the highest values




Different usecases:

1. Text completion

2. Question answering

3. Summarisation

4. Translation

# 1. Text Completion

Feed in some random text and see what the AI generates from that!

**Example usage**
 !python3 src/interactive_conditional_samples.py --nsamples=2 --top_k=40 --temperature=.80 --model_name='345M'

Model prompt >>> "Our solar system consists of the inner and outer planets, separated by an asteroid belt. It has "

Model prompt >>> "The 10 best foods are: 1. Peanut butter jelly sandwiches 2. Marshmallows 3. Broccoli 4."

Try it out yourself!

In [8]:

!python3 src/interactive_conditional_samples.py --nsamples=2 --top_k=40 --temperature=.80 --model_name='345M'

W0731 14:08:53.934029 140006535784320 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:56: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-07-31 14:08:53.935909: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-31 14:08:53.938920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-31 14:08:53.939333: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-07-31 14:08:53.939644: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-31 14:08:53.940913: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Succe

# 2. Question-Answering
Feed in a passage, and then some question/answer pairs (Q: blah blah? A: Blah blah.), and token A:. The AI will answer the previous ''Q:''

Note, for a single word answer (i.e.: Yes/No, city), set flag length=1

**Example usage**

> !python3 src/interactive_conditional_samples.py --nsamples=10 --top_k=40 --temperature=.80 --length=1 --model_name='345M'
>
>Model Prompt >>>
>
>The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer
Olympics, with the theme of “one world, one dream”. Plans for the relay were announced on April 26, 2007, in
Beijing, China. The relay, also called by the organizers as the “Journey of Harmony”, lasted 129 days and carried
the torch 137,000 km (85,000 mi) – the longest distance of any Olympic torch relay since the tradition was started
ahead of the 1936 Summer Olympics.
After being lit at the birthplace of the Olympic Games in Olympia, Greece on March 24, the torch traveled to the Panathinaiko Stadium in Athens, and then to Beijing, arriving on March 31. From Beijing, the torch was
following a route passing through six continents. The torch has visited cities along the Silk Road, symbolizing
ancient links between China and the rest of the world. The relay also included an ascent with the flame to the top of
Mount Everest on the border of Nepal and Tibet, China from the Chinese side, which was closed specially for the event.
>
>Q: What was the length of the race?
>
>A: 137,000 km
>
>Q: Was it larger than previous ones?
>
>A: No
>
>Q: Where did the race begin?
>
>A: Olympia, Greece
>
>Q: Where did they go after?
>
>A: Athens
>
>Q: How many days was the race?
>
>A: seven
>
>Q: Did they visit any notable landmarks?
>
>A: Panathinaiko Stadium
>
>Q: And did they climb any mountains?
>
>A:






In [10]:
!python3 src/interactive_conditional_samples.py --nsamples=10 --top_k=40 --temperature=.80 --length=1 --model_name='345M'

W0731 14:26:23.855015 140543752943488 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:56: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-07-31 14:26:23.856928: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-31 14:26:23.859796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-31 14:26:23.860230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-07-31 14:26:23.860504: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-31 14:26:23.861834: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Succe

# 3. Summarization

Feed in a passage, and add and text TL;DR: or Summary: at the end, and the AI will try to summarise the text.

**Example usage**

Note the following passage was obtained from a blog post about the mars-water paradox.

> !python3 src/interactive_conditional_samples.py --nsamples=3 --length=100 --temperature=1 --model_name='345M'
>
>Model Prompt >>>
>
>Mars has been the most extensively studied planet in the Solar System, except of course Earth. For the last 25 years, these missions have focused on the search for life by “following the water.” Although we have acquired compelling evidence of flowing liquid water on early Mars, the fundamental question about how water could be stable under Martian atmospheric conditions remains unsolved. Everything we have learned about Mars points towards a freezing cold Martian climate that would be incapable of stabilizing liquid water throughout Mars’ history.
The two ideas that suggest liquid water could not be stable on early Mars are the “Faint Young Sun Paradox” and the Martian orbit. The following is a summary of two recent papers about the problem of Mars’ early climate: “The climate of early Mars,” by Robin Wordsworth, and a book chapter by Robert Haberle and coauthors, “The Early Mars Climate System.” Mars today as we know it is a cold and dry desert with a thin atmosphere not capable of stabilizing liquid water on its surface. However, there is ample evidence that Mars had flowing liquid water on its surface about 4 to 3.7 billion years ago (named as the Noachian Period). The evidence gathered by Mars orbiters, rovers, and landers is geomorphological; (valley networks, crater lakes, purported Northern ocean, glacial landforms, etc.); mineralogical (iron- and magnesium-rich clay minerals, sulfates, chlorides, iron oxides, and oxyhydroxides, etc.); and isotopic (noble gases, nitrogen, hydrogen, oxygen and carbon).
TL;DR:
Model Prompt >>>
>
>Theodore McCarrick is the most senior Catholic figure to be dismissed from the priesthood in modern times.
US Church officials said allegations he had sexually assaulted a teenager five decades ago were credible.
Mr McCarrick, 88, had previously resigned but said he had "no recollection" of the alleged abuse.
"No bishop, no matter how influential, is above the law of the Church," Cardinal Daniel DiNardo, president of the United States Conference of Catholic Bishops said in a statement.
"For all those McCarrick abused, I pray this judgment will be one small step, among many, toward healing."
The alleged abuses may have taken place too long ago for criminal charges to be filed because of the statute of limitations.
Mr McCarrick was the archbishop of Washington DC from 2001 to 2006. Since his resignation last year from the College of Cardinals, he has been living in seclusion in a monastery in Kansas.
He was the first person to resign as a cardinal since 1927.
He is among hundreds of members of the clergy accused of sexually abusing children over several decades and his dismissal comes days before the Vatican hosts a summit on preventing child abuse.
The Vatican said Pope Francis had ruled Mr McCarrick's expulsion from the clergy as definitive, and would not allow any further appeals against the decision. 
TL;DR:





In [11]:
!python3 src/interactive_conditional_samples.py --nsamples=3 --length=100 --temperature=1 --model_name='345M'

W0731 14:33:51.335853 140626288285568 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:56: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-07-31 14:33:51.337776: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-31 14:33:51.340773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-31 14:33:51.341222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-07-31 14:33:51.341511: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-31 14:33:51.342917: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Succe

# 4. Translation
Provide a few example translations, using the format of english_text = other_language_text, and then english_text = at the end. The AI will try to translate the english text into the other language.

**Example usage**

>!python3 src/interactive_conditional_samples.py --nsamples=3 --temperature=1 --model_name='345M'
>
>Model Prompt >>>
>
>Good morning. = Buenos días.
I am lost. Where is the restroom? = Estoy perdido. ¿Dónde está el baño?
How much does it cost? = ¿Cuánto cuesta?
How do you say maybe in Spanish? = ¿Cómo se dice maybe en Español?
Would you speak slower, please. = Por favor, habla mas despacio.
Where is the book store? = ¿Dónde está la librería?
At last a feminist comedian who makes jokes about men. = Por fin un cómico feminista que hace chistes sobre hombres.
How old are you? =
Try it out yourself!

In [12]:
!python3 src/interactive_conditional_samples.py --nsamples=3 --temperature=1 --model_name='345M'

W0731 14:37:21.173357 139647590905728 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:56: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-07-31 14:37:21.175250: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-31 14:37:21.178265: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-31 14:37:21.178668: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-07-31 14:37:21.178975: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-31 14:37:21.180349: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Succe